In [1]:
#Mounting Colab with GDrive
from google.colab import drive
drive.mount('/content/drive')
import os

import numpy as np
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVC
from sklearn.metrics import classification_report

import os
os.chdir('drive/MyDrive/DAIC')

Mounted at /content/drive


In [2]:
def makedata(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    X = np.array(X)
    X = np.average(X, axis = 0)
    X = np.array(X.reshape(1, -1))
    return X

dev = np.array(pd.read_csv('/content/drive/MyDrive/DAIC/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
test = np.array(pd.read_csv('/content/drive/MyDrive/DAIC/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
train = np.array(pd.read_csv('/content/drive/MyDrive/DAIC/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]

X_train = np.zeros((1,74))
Y_train = []

X_test = np.zeros((1,74))
Y_test = []

for i in range(len(test)):
    data = pd.read_csv('/content/drive/MyDrive/DAIC/test_data/'+str(int(test[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    X_test = np.concatenate((X_test,X_temp),0)
    Y_test.append(test[i][1])
    

X_test = np.delete(X_test,0,0)
Y_test = np.array(Y_test)
count_0 = 0
count_1 = 0

for i in range(len(train)):
    data = pd.read_csv('/content/drive/MyDrive/DAIC/train_data/'+str(int(train[i][0]))+"_COVAREP.csv",header = None)
    if(train[i][1] == 0):
      count_0 +=1
      if(count_0<27):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(train[i][1])
    else:
      count_1 +=1
      if(count_1<27):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(train[i][1])

count_0 = 0
count_1 = 0    

for i in range(len(dev)):
    data = pd.read_csv('/content/drive/MyDrive/DAIC/dev_data/'+str(int(dev[i][0]))+"_COVAREP.csv",header = None)
    if(dev[i][1] == 0):
      count_0 += 1
      if(count_0<27):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(dev[i][1])
    else:
      count_1+= 1
      if(count_1<27):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(dev[i][1])
    

X_train = np.delete(X_train,0,0)
Y_train = np.array(Y_train)

# Running Models

In [6]:
class RfAudio:
  def __init__(self):
    self.classifier = RandomForestClassifier()
  
  def fitModel(self, X, Y):
    self.classifier.fit(X, Y)
  
  def predictModel(self, X):
    return self.classifier.predict(X)
  
  def scoreModel(self, X, Y):
    return self.classifier.score(X, Y)

class SVMAudio:
  def __init__(self, kernel = "rbf"):
    self.classifier = SVC(kernel = kernel, random_state = 0)
  
  def fitModel(self, X, Y):
    self.classifier.fit(X, Y)
  
  def predictModel(self, X):
    return self.classifier.predict(X)
  
  def scoreModel(self, X, Y):
    return self.classifier.score(X, Y)


rfModel = RfAudio()
rfModel.fitModel(X_train, Y_train)
Y_pred1  = rfModel.classifier.predict(X_test)
print("Random forest: ")
print(classification_report(Y_test, Y_pred1))

svmModel = SVMAudio()
svmModel.fitModel(X_train, Y_train)
Y_pred2  = svmModel.classifier.predict(X_test)
print("SVM: ")
print(classification_report(Y_test, Y_pred2))

Random forest: 
              precision    recall  f1-score   support

           0       0.72      0.64      0.68        33
           1       0.33      0.43      0.38        14

    accuracy                           0.57        47
   macro avg       0.53      0.53      0.53        47
weighted avg       0.61      0.57      0.59        47

SVM: 
              precision    recall  f1-score   support

           0       0.61      0.33      0.43        33
           1       0.24      0.50      0.33        14

    accuracy                           0.38        47
   macro avg       0.43      0.42      0.38        47
weighted avg       0.50      0.38      0.40        47

